In [1]:
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from torch import Tensor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
concate_df1 = []
concate_df2 = []
concate_df3 = []
#concate_df4 = []

for i in range(1,61):
    filename1 = '../data/csv_file/class1_run'+str(i)+'.csv'
    filename2 = '../data/csv_file/class2_run'+str(i)+'.csv'
    filename3 = '../data/csv_file/class3_run'+str(i)+'.csv'
    #filename4 = '../data/csv_file/class4_run'+str(i)+'.csv'
    
    df1 = pd.read_csv(filename1,header=None)
    df2 = pd.read_csv(filename2,header=None)
    df3 = pd.read_csv(filename3,header=None)
    #df4 = pd.read_csv(filename4,header=None)
    
    concate_df1.append(df1)
    concate_df2.append(df2)
    concate_df3.append(df3)
    #concate_df4.append(df4)
    
sum_df1 = pd.concat(concate_df1, axis=1)
sum_df2 = pd.concat(concate_df2, axis=1)
sum_df3 = pd.concat(concate_df3, axis=1)
#sum_df4 = pd.concat(concate_df4, axis=1)
#print(sum_df1.shape,sum_df2.shape,sum_df3.shape,sum_df4.shape) 
#X = pd.concat([sum_df1,sum_df2,sum_df3,sum_df4],axis=1)
X = pd.concat([sum_df1,sum_df2,sum_df3],axis=1)

print(X.shape,sum_df1.shape, sum_df2.shape, sum_df3.shape)

(64, 863360) (64, 291200) (64, 284800) (64, 287360)


In [3]:
#extract the class 4 data
concate_df4 = []

for i in range(1,21):
    filename4 = '../data/csv_file/class4_run'+str(i)+'.csv'
    df4 = pd.read_csv(filename4,header=None)
    concate_df4.append(df4)
    
sum_df4 = pd.concat(concate_df4, axis=1)    
print(sum_df4.shape)

output = []

# maually design 450 samples for class 4 
for i in range(450):
    offset = np.random.randint(0,sum_df4.shape[1]-640)
    output.append(sum_df4.to_numpy()[:,offset:offset+640]) 

output_df4 = np.hstack(output)
print(output_df4.shape)

X = np.concatenate((X.to_numpy(), output_df4), axis=1)

print(X.shape)

(64, 192000)
(64, 288000)
(64, 1151360)


In [4]:
#average
X_average = X - np.mean(X,0)
print(X_average.shape)

(64, 1151360)


In [5]:
#standardization trail-wise
trial_n = int(X_average.shape[1]/640)
X_temp = X_average.reshape(64,trial_n,640)
print(X_temp.shape)

for i in range(64):
    for j in range(640):
        m = X_temp[i,:,j]
        m = (m - m.mean())/m.std()
        #print(m.mean(),m.std())
        X_temp[i,:,j] = m

X_normal = X_temp
X_normal = X_normal.reshape(64,-1)

print(X_normal.shape)

(64, 1799, 640)
(64, 1151360)


In [6]:
#cut to 640 
#samples in each class 455 445 449 450
empty_list = []
for i in range(int(X_normal.shape[1]/640)):
    temp = X_normal[:,640*i:640*(i+1)]
    temp = temp.reshape(1,-1)
    empty_list.append(temp)
    

X = np.vstack(empty_list)
print(X.shape)

(1799, 40960)


In [7]:
#make label 
class1_label = np.zeros(int(sum_df1.shape[1]/640))
class2_label = np.zeros(int(sum_df2.shape[1]/640)) + 1
class3_label = np.zeros(int(sum_df3.shape[1]/640)) + 2
class4_label = np.zeros(int(output_df4.shape[1]/640)) + 3

Y = np.concatenate((class1_label,class2_label,class3_label,class4_label),axis=0)
print(Y.shape)

(1799,)


In [8]:
train_set, test_set, train_label, test_label = train_test_split( X, Y, test_size=0.2, random_state=42)
print(train_set.shape, train_label.shape, test_set.shape, test_label.shape)

(1439, 40960) (1439,) (360, 40960) (360,)


In [9]:
pd.DataFrame(train_set).to_csv("../data/64_640data/train_set.csv",index = False)
pd.DataFrame(train_label).to_csv("../data/64_640data/train_label.csv",index = False)
pd.DataFrame(test_set).to_csv("../data/64_640data/test_set.csv",index = False)
pd.DataFrame(test_label).to_csv("../data/64_640data/test_label.csv",index = False)